<a href="https://colab.research.google.com/github/usshaa/SMBDA/blob/main/C-3.2%3A%20MapReduce_Handson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**MapReduce (Basic Job Execution)**
###What We will Learn in this :
####how to run a MapReduce job, including creating and executing jobs on HDFS.




# Install Java Development Kit (JDK

In [1]:
!sudo apt-get update
!sudo apt-get install openjdk-8-jdk
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,369 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,666 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,533 kB]
Hit:13 https://ppa.launchpadcontent.

# setup Hadoop as told in previous class

In [2]:
# Download Hadoop
!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz

# Extract Hadoop
!tar -xzf hadoop-3.3.6.tar.gz

# setting up environment

In [3]:
# Set Hadoop environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
os.environ["PATH"] = os.environ["PATH"] + ":" + os.environ["HADOOP_HOME"] + "/bin" + ":" + os.environ["HADOOP_HOME"] + "/sbin"

# Configure Hadoop (replace with your desired configuration)
!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64" >> ~/.bashrc
!echo "export HADOOP_HOME=/content/hadoop-3.3.6" >> ~/.bashrc
!echo "export PATH=$PATH:$HADOOP_HOME/bin:$HADOOP_HOME/sbin" >> ~/.bashrc

# Source the bashrc file to apply the changes
!source ~/.bashrc

# Verify Hadoop installation
!hadoop version

Hadoop 3.3.6
Source code repository https://github.com/apache/hadoop.git -r 1be78238728da9266a4f88195058f08fd012bf9c
Compiled by ubuntu on 2023-06-18T08:22Z
Compiled on platform linux-x86_64
Compiled with protoc 3.7.1
From source with checksum 5652179ad55f76cb287d9c633bb53bbd
This command was run using /content/hadoop-3.3.6/share/hadoop/common/hadoop-common-3.3.6.jar


# verfying the working of hadoop

In [27]:
!jps

3355 Jps


# so initially will create an input.txt file

In [28]:
!hdfs dfs -mkdir -p /content/new_folder

In [29]:
with open('/content/new_folder/input.txt', 'w') as f:
    f.write("Hello HadoopHadoop MapReduce is great for big dataBig data can be processed using Hadoop MapReduce is a programming model")

In [30]:
!hadoop jar /content/hadoop-3.3.6/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.6.jar wordcount /content/new_folder/input.txt /content/new_folder/output

2025-03-07 07:23:13,924 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-03-07 07:23:13,995 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-03-07 07:23:13,995 INFO impl.MetricsSystemImpl: JobTracker metrics system started
org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/content/new_folder/output already exists
	at org.apache.hadoop.mapreduce.lib.output.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:164)
	at org.apache.hadoop.mapreduce.JobSubmitter.checkSpecs(JobSubmitter.java:277)
	at org.apache.hadoop.mapreduce.JobSubmitter.submitJobInternal(JobSubmitter.java:143)
	at org.apache.hadoop.mapreduce.Job$11.run(Job.java:1678)
	at org.apache.hadoop.mapreduce.Job$11.run(Job.java:1675)
	at java.security.AccessController.doPrivileged(Native Method)
	at javax.security.auth.Subject.doAs(Subject.java:422)
	at org.apache.hadoop.security.UserGroupInformation.doAs(UserGroupInformation.java:1899)
	a

In [31]:
#displaying the result of MapReduce WordCount Program
!hdfs dfs -cat /content/new_folder/output/part-r-00000

Hadoop	1
HadoopHadoop	1
Hello	1
MapReduce	2
a	1
be	1
big	1
can	1
data	1
dataBig	1
for	1
great	1
is	2
model	1
processed	1
programming	1
using	1


**So here we have seen excution of MapReduce using Existing jar Files**

# Working On New dataset using own Scripting Of map Reduce

**Case Study 2: Titanic Dataset - Survival Analysis**
**Dataset Source:**
Titanic Dataset

**Task:**
Count the number of survivors and non-survivors.

**Steps:**
Prepare the Data: Download and upload



1. the train.csv file to /content/newfolder.

2. Write the MRJob Script: Save this as survival_analysis.py.

### downloading the dataset

In [32]:
#!/bin/bash
!kaggle datasets download hesh97/titanicdataset-traincsv

Dataset URL: https://www.kaggle.com/datasets/hesh97/titanicdataset-traincsv
License(s): CC0-1.0
titanicdataset-traincsv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [33]:
!unzip titanicdataset-traincsv.zip -d /content/new_folder

Archive:  titanicdataset-traincsv.zip
replace /content/new_folder/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/new_folder/train.csv  


### viewing the dataset

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


df= pd.read_csv('/content/new_folder/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### writing the script to count the passenger

In [35]:
!pip install mrjob

In [42]:
%%writefile survival_analysis.py
from mrjob.job import MRJob

class SurvivalAnalysis(MRJob):

    def mapper(self, _, line):
        fields = line.split(',')
        if fields[0] != "PassengerId":  # Skip the header
            survived = fields[1]
            yield survived, 1

    def reducer(self, survived, counts):
        yield survived, sum(counts)

if __name__ == "__main__":
    SurvivalAnalysis.run()


Overwriting survival_analysis.py


# Run the Script:

In [43]:
!python3 survival_analysis.py /content/new_folder/train.csv > /content/new_folder/survival_analysis_output.txt


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/survival_analysis.root.20250307.072926.410647
Running step 1 of 1...
job output is in /tmp/survival_analysis.root.20250307.072926.410647/output
Streaming final output from /tmp/survival_analysis.root.20250307.072926.410647/output...
Removing temp directory /tmp/survival_analysis.root.20250307.072926.410647...


#View the Results:

In [44]:
!cat /content/new_folder/survival_analysis_output.txt


"1"	342
"0"	549


here we have found that 342 paasenger survived where as 549 not